In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import geobr
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 250);pd.set_option('display.max_rows', 250)
import os


#### Funções

In [2]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    
    
    
def build_statistics_metrics(ano_enem : int, 
                             lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(ano_enem, lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(ano_enem, lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df


def build_distribution_freq(df_enem, cols_to_group, cat_columns):
    
    def get_distribution_freq(df_enem, cols_to_group, cat_column):
        agg_result = (pd.get_dummies(df_enem[cols_to_group+[cat_column]],
               columns = [cat_column])
                 .groupby(cols_to_group).sum()
                )
        return agg_result
    
    result    = [get_distribution_freq(df_enem, cols_to_group, cat_column) for cat_column in cat_columns]
    result_df = (pd.concat(result, axis = 1))
    return result_df

def get_count(ano_enem, df_enem, cols_to_group, column, column_name = None):
    agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group)
         .agg({column:'count'})
         .rename(columns = {column : column_name} )
        )
    return agg_result

#### Notebook

In [2]:
os.chdir('C:\\Users\\Bruno\\OneDrive\\Área de Trabalho\\Microdados\\Datas')

#### Casa

In [3]:
os.chdir('D:\Trabalho\Microdados\ENEM')

In [3]:
pwd

'D:\\Trabalho\\Microdados\\ENEM'

#### Repositório Casa

In [4]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem.xlsx')

In [4]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem6.xlsx')

#### Repositório Notebook

In [3]:
df_enem = pd.read_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem.xlsx')

In [3]:
df_enem = pd.read_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem6.xlsx')

#### 1º Ajuste - Dados de 2010 até 2022 - testar

In [5]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem2.xlsx')

In [4]:
df_enem = pd.read_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem2.xlsx')

In [6]:
df_enem.shape

(13000, 189)

In [6]:
df_enem.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_CERTIFICADO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,TP_ANO_CONCLUIU,TP_ESCOLA,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050,Q051,Q052,Q053,Q054,Q055,Q056,Q057,Q058,Q059,Q060,Q061,Q062,Q063,Q064,Q065,Q066,Q067,Q068,Q069,Q070,Q071,Q072,Q073,Q074,Q075,TP_NACIONALIDADE,ANO_CONCLUIU,Q076,IN_TREINEIRO
0,200000000001,2010,9,M,0.0,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3143302,MONTES CLAROS,31,MG,1,1,1,1,91.0,87.0,94.0,98.0,675.0,787.7,698.0,697.0,BEBEDACDBABADADCBABBADDEDEBBCDBCBDADADEEDCDCE,AABBDACBDADECCBEAACDCDBBCCEDECBBDBDAABEEDDCED,DADDCCECEEACDECDBDABCCEEBEAADACDBCEDDBDBBAAAD,CEEBDBACBABBEBAACDDCBCBDEDDCACEDBDBBDBBCDEBDD,0.0,CABEAACDCEBADADCEBBBADDEAEEBDDCCBDBDAEEADCDCE,AABBDACBCABECCBEBACACDBBCCEDECEBDEDAABEEDDCED,DAEDCDDEADCECEEACDECDBDABCCBEBEDABACDBCEDDADDB...,CEEBDCACBABAEBBAEDCDCDAECBDBDECBBDBBCBDEDEBDE,P,875.0,625.0,625.0,500.0,625.0,650.0,A,C,C,A,A,A,A,A,5.0,0.0,3.0,2.0,3.0,B,B,A,A,B,B,B,B,C,B,4.0,5.0,0.0,0.0,B,A,A,B,A,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000002,2010,7,F,0.0,4,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1721000,PALMAS,17,TO,1,1,1,1,90.0,86.0,94.0,98.0,504.2,548.2,431.4,415.9,CAEEBBCDAEBDEDBCDCEABBAADDAADDDBACDEABDABCADB,BCDBBDEBEBBEEDCBCADDEBCAECBEADDAEBCAADBAAECDE,EEECBBEEB*ECABEDAEACCCCCCCACDBCDDABABCEEBEECB,CBAECBABCCEADDDDDDDDDDDDDBCDCCEBABCDEDCBABCDE,1.0,ACAEBCCEADADDBACBEBABDEEADEDDBCBCDDBAEEACDDEC,BAABADCBACBECACEBBCCBDCACDCEBDEDBEEEADBEAECDD,DAEDCDDEADCECEEACDECDBDABCCBEBEDABACDBCEDDADDB...,CEEBDCACBABAEBBAEDCDCDAECBDBDECBBDBBCBDEDEBDE,P,500.0,375.0,375.0,250.0,250.0,350.0,E,I,D,A,A,B,B,A,5.0,0.0,5.0,5.0,5.0,B,B,B,B,B,B,B,B,D,A,5.0,5.0,0.0,5.0,B,A,A,B,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000003,2010,12,M,1.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1302603,MANAUS,13,AM,1,1,1,1,90.0,86.0,96.0,100.0,517.4,562.0,563.6,463.1,CBAEBACEDBABDDAEECEBEECDCBBBCCBBCCDCECDECDDCB,BACBCDDBACDEEEECADEECDCDCEEECBDDDAEBACBAAECDA,EAADCAEAECAAACCDBDACCCCCBCABAACDECDEBEEDEBAAA,CEAADDDBCAEDCBAADBDDCCBCCCBBACCCAEDCCBCACBBBD,0.0,ACAEBCCEADADDBACBEBABDEEADEDDBCBCDDBAEEACDDEC,BAABADCBACBECACEBBCCBDCACDCEBDEDBEEEAD

In [7]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    

In [8]:
# DATA PREP
notas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()
lista_provas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()

df_enem = (df_enem
           .query('IN_TREINEIRO != 1')
           .drop(columns = df_enem.columns[df_enem.columns.str.contains('TX_')].tolist()+['IN_TREINEIRO'])
          )


# REGIÃO GEOGRÁFICA
df_enem['TP_REGIAO_PROVA'] = df_enem.SG_UF_PROVA.apply(create_columns_region)
df_enem['TP_REGIAO_ESC'] = df_enem.SG_UF_PROVA.apply(create_columns_region)

# ABSTENÇÃO - PRIMEIRO DIA
df_enem.loc[(df_enem['TP_PRESENCA_CH'] == 0) & (df_enem['TP_PRESENCA_LC'] == 0), ['ABS_DIA_1']] = 1
df_enem.loc[(df_enem['TP_PRESENCA_CH'] != 0) & (df_enem['TP_PRESENCA_LC'] != 0), ['ABS_DIA_1']] = 0
# ABSTENÇÃO - SEGUNDO DIA
df_enem.loc[(df_enem['TP_PRESENCA_CN'] == 0) & (df_enem['TP_PRESENCA_MT'] == 0), ['ABS_DIA_2']] = 1
df_enem.loc[(df_enem['TP_PRESENCA_CN'] != 0) & (df_enem['TP_PRESENCA_MT'] != 0), ['ABS_DIA_2']] = 0
# ABSTENÇÃO FULL DAYS
df_enem.loc[(df_enem['ABS_DIA_1'] == 1) & (df_enem['ABS_DIA_2'] == 1), ['ABS_ALL']] = 1
df_enem.loc[(df_enem['ABS_DIA_1'] != 1) & (df_enem['ABS_DIA_2'] != 1), ['ABS_ALL']] = 0

#Acrescentei
df_enem.loc[((df_enem['ABS_DIA_1'] == 0) | (df_enem['ABS_DIA_2'] == 0)), ['ABS_ALL']] = 0

# FLAG ESCOLA NULL
df_enem.loc[df_enem['NO_MUNICIPIO_ESC'].isna(), ['FLAG_NULL_ESC']]  = 1
df_enem.loc[df_enem['NO_MUNICIPIO_ESC'].notna(), ['FLAG_NULL_ESC']] = 0

# MÉDIAS
df_enem['MEDIA_OBJ'] = df_enem[[prova for prova in notas if prova != 'NU_NOTA_REDACAO']].mean(axis = 1)
df_enem['MEDIA_GLOBAL'] = df_enem[notas].mean(axis = 1)

In [9]:
df_enem.shape

(12671, 188)

In [31]:
df_enem.to_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem1.xlsx', index=False)

#### 2º Ajuste - Abstenções com novo df maior

In [10]:
df_enem.to_excel('C:\\Users\\Bruno\\PYTHON\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem3.xlsx', index=False)

In [11]:
geral = df_enem['NU_ANO'].value_counts().reset_index()
geral.rename(columns={'index': 'NU_ANO', 'NU_ANO': 'QTD_INSCRITOS'}, inplace=True)
geral = geral.sort_values(by= 'NU_ANO' , ascending=True)
geral.head(13)

,NU_ANO,QTD_INSCRITOS
0,2010,1000
1,2011,1000
2,2012,1000
3,2013,1000
4,2014,1000
8,2015,956
7,2016,964
5,2017,982
9,2018,953
6,2019,978


#### Abstenção

In [12]:
geral1 = df_enem.groupby('NU_ANO')['ABS_ALL'].sum().reset_index()
geral1.head(13)


,NU_ANO,ABS_ALL
0,2010,243.0
1,2011,261.0
2,2012,271.0
3,2013,349.0
4,2014,313.0
5,2015,221.0
6,2016,282.0
7,2017,294.0
8,2018,209.0
9,2019,263.0


In [13]:
geral = geral.merge(geral1, on='NU_ANO', how='left')
geral.head()

,NU_ANO,QTD_INSCRITOS,ABS_ALL
0,2010,1000,243.0
1,2011,1000,261.0
2,2012,1000,271.0
3,2013,1000,349.0
4,2014,1000,313.0


##### Proporção Inscritos

In [14]:
geral['ABS_ALL_%'] = ((geral['ABS_ALL'] / geral['QTD_INSCRITOS'])*100).round(2)
geral.head(13)

,NU_ANO,QTD_INSCRITOS,ABS_ALL,ABS_ALL_%
0,2010,1000,243.0,24.30
1,2011,1000,261.0,26.10
2,2012,1000,271.0,27.10
3,2013,1000,349.0,34.90
4,2014,1000,313.0,31.30
5,2015,956,221.0,23.12
6,2016,964,282.0,29.25
7,2017,982,294.0,29.94
8,2018,953,209.0,21.93
9,2019,978,263.0,26.89


#### Caso eu queira inserir o %

In [37]:
geral['ABS_ALL_%'] = geral['ABS_ALL_%'].astype(str) + '%'
geral.head()

,NU_ANO,QTD_INSCRITOS,ABS_ALL,ABS_ALL_%
0,2018,953,209.0,21.93%
1,2019,978,263.0,26.89%


#### Média por ano.

In [7]:
geral1_media = df_enem.groupby('NU_ANO')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral1_media.head(13)

,NU_ANO,MEDIA_OBJ,MEDIA_GLOBAL
0,2010,538.63,412.03
1,2011,526.11,392.07
2,2012,527.54,531.68
3,2013,484.62,309.50
4,2014,523.17,353.86
5,2015,535.45,415.85
6,2016,554.62,566.95
7,2017,546.05,551.97
8,2018,529.25,527.63
9,2019,509.18,524.51


#### Tipo de língua
#### 0 - Inglês
#### 1 - Espanhol

In [15]:

geral2 = df_enem.groupby(['NU_ANO', 'TP_LINGUA'])['NU_INSCRICAO'].count().reset_index()


geral2 = geral2.pivot(index='NU_ANO', columns='TP_LINGUA', values='NU_INSCRICAO').reset_index()


geral2.columns = ['NU_ANO', 'QTD_INGLES', 'QTD_ESPANHOL']
geral2.head(13)

,NU_ANO,QTD_INGLES,QTD_ESPANHOL
0,2010,463,537
1,2011,385,615
2,2012,283,424
3,2013,382,618
4,2014,446,554
5,2015,464,492
6,2016,535,429
7,2017,508,474
8,2018,426,527
9,2019,455,523


#### Proporção língua

In [17]:
geral2_pct = geral2[['QTD_INGLES','QTD_ESPANHOL']]
geral2_pct.head()

,QTD_INGLES,QTD_ESPANHOL
0,463,537
1,385,615
2,283,424
3,382,618
4,446,554


In [19]:
soma_total = geral2_pct.sum(axis=1)

# Iterar sobre as colunas e criar as colunas percentuais automaticamente
for coluna in geral2_pct.columns:
    if coluna.startswith('QTD_'):  # Certifica-se de que estamos lidando apenas com as colunas relevantes
        coluna_percentual = coluna + '_%'
        proporcao_coluna = geral2_pct[coluna] / soma_total
        geral2_pct[coluna_percentual] = (proporcao_coluna * 100).round(2)

geral2_pct.head()

geral2 = geral2.merge(geral2_pct, on=['QTD_INGLES','QTD_ESPANHOL'], how='outer')
geral2.head(13)

,NU_ANO,QTD_INGLES,QTD_ESPANHOL,QTD_INGLES_%,QTD_ESPANHOL_%
0,2010,463,537,46.30,53.70
1,2011,385,615,38.50,61.50
2,2012,283,424,40.03,59.97
3,2013,382,618,38.20,61.80
4,2014,446,554,44.60,55.40
5,2015,464,492,48.54,51.46
6,2016,535,429,55.50,44.50
7,2017,508,474,51.73,48.27
8,2018,426,527,44.70,55.30
9,2019,455,523,46.52,53.48


#### Média por língua.

In [9]:
geral2_media = df_enem.groupby('TP_LINGUA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral2_media['TP_LINGUA'] = geral2_media['TP_LINGUA'].map({0: 'Inglês', 1: 'Espanhol'})
geral2_media.head(13)

,TP_LINGUA,MEDIA_OBJ,MEDIA_GLOBAL
0,Inglês,542.36,485.35
1,Espanhol,507.17,439.53


#### Unir com df geral

In [20]:
geral = geral.merge(geral2, on='NU_ANO', how='left')
geral.head()

,NU_ANO,QTD_INSCRITOS,ABS_ALL,ABS_ALL_%,QTD_INGLES,QTD_ESPANHOL
0,2018,953,209.0,21.93,426,527
1,2019,978,263.0,26.89,455,523


#### Por tipo de Escola
#### 1 - Não respondeu
#### 2 - Escola pública
#### 3 - Escola Privada
#### 4 - Exterior
#### Aqui tem uma observação esse modelo do dicionários não foi seguido nos anos de 2012 a 2014, o ano de 2010 nem possui esse item. O modelo de 2012 a 2014 segue:
#### 1 - Escola pública
#### 2 - Escola privada

#### Recorte para ajustar os anos que houve mudança.

In [11]:
df_enem.loc[df_enem['NU_ANO'].between(2012, 2014), 'TP_ESCOLA'] = df_enem['TP_ESCOLA'] + 1
df_enem[df_enem['NU_ANO'] == 2012]

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_CERTIFICADO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,TP_ANO_CONCLUIU,TP_ESCOLA,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050,Q051,Q052,Q053,Q054,Q055,Q056,Q057,Q058,Q059,Q060,Q061,Q062,Q063,Q064,Q065,Q066,Q067,Q068,Q069,Q070,Q071,Q072,Q073,Q074,Q075,TP_NACIONALIDADE,ANO_CONCLUIU,Q076,TP_REGIAO_PROVA,TP_REGIAO_ESC,ABS_DIA_1,ABS_DIA_2,ABS_ALL,FLAG_NULL_ESC,MEDIA_OBJ,MEDIA_GLOBAL
2000,400000000001,2012,13,M,1.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2933307,VITORIA DA CONQUISTA,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,NaN,NaN,NaN,NaN,I,B,D,3,A,B,A,A,D,A,D,D,A,D,D,B,A,A,A,D,A,B,4,5,0,5,5,5,5,E,D,A,D,C,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,1.0,1.0,1,1,NaN,NaN
2001,400000000002,2012,3,M,0.0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2611606,RECIFE,26,PE,1,1,1,1,141.0,137.0,146.0,150.0,537.7,552.2,532.6,468.3,1.0,P,80.0,60.0,60.0,80.0,0.0,280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,E,C,G,4,A,B,A,A,A,B,A,A,A,D,A,B,A,A,D,D,A,C,5,1,0,5,2,5,0,A,A,D,B,A,A,A,B,B,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,0.0,0.0,0,1,522.700,474.16
2002,400000000003,2012,5,F,0.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3301702,DUQUE DE CAXIAS,33,RJ,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,C,C,C,4,B,B,B,A,D,D,D,A,A,D,D,B,D,A,D,D,A,A,3,5,1,0,0,5,3,A,A,A,A,B,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sudeste,Sudeste,1.0,1.0,1,1,NaN,NaN
2003,400000000004,2012,11,M,0.0,3,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5300108,BRASILIA,53,DF,1,1,1,1,143.0,139.0,147.0,151.0,459.0,557.7,545.0,565.3,1.0,P,140.0,160.0,140.0,160.0,120.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,NaN,NaN,NaN,NaN,C,C,F,4,A,B,B,A,D,D,A,D,A,D,A,C,A,A,D,D,B,A,5,5,0,5,5,5,

#### 4º Ajuste - Salvando nova alteração de dataset, ajustando os anos diferentes. Casa

In [12]:
df_enem.to_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem5.xlsx', index=False)

#### Carregar. Casa

In [ ]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem5.xlsx', sheet_name='Sheet1')

#### Carregar notebook.

In [5]:
df_enem = pd.read_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem5.xlsx')

In [28]:
geral3 = df_enem.groupby(['NU_ANO', 'TP_ESCOLA'])['NU_INSCRICAO'].count().reset_index()


geral3 = geral3.pivot(index='NU_ANO', columns='TP_ESCOLA', values='NU_INSCRICAO').reset_index()


geral3.columns = ['NU_ANO', 'QTD_NAO_RESP','QTD_PUBLICA','QTD_PRIVADA','QTD_EXTERIOR']
geral3 = geral3.fillna(0).astype(int) # Possui dados faltantes e possui a falta da variável 'QTD_EXTERIOR' em alguns anos.
geral3.head(13)

,NU_ANO,QTD_NAO_RESP,QTD_PUBLICA,QTD_PRIVADA,QTD_EXTERIOR
0,2011,752,188,60,<NA>
1,2012,<NA>,66,14,<NA>
2,2013,<NA>,155,24,<NA>
3,2014,<NA>,740,138,<NA>
4,2015,884,52,20,<NA>
5,2016,905,37,22,<NA>
6,2017,881,78,19,4
7,2018,750,175,3,25
8,2019,778,199,1,<NA>
9,2020,731,212,8,<NA>


#### Proporção escolas

In [7]:
geral3_pct = geral3[['QTD_PUBLICA','QTD_PRIVADA','QTD_EXTERIOR']]
geral3_pct.head(13)

,QTD_PUBLICA,QTD_PRIVADA,QTD_EXTERIOR
0,188,60,0
1,66,14,0
2,155,24,0
3,740,138,0
4,52,20,0
5,37,22,0
6,78,19,4
7,175,3,25
8,199,1,0
9,212,8,0


#### Das que foram respondidas a proporção segue a distribuição abaixo.

In [8]:
soma_total = geral3_pct.sum(axis=1)

# Iterar sobre as colunas e criar as colunas percentuais automaticamente
for coluna in geral3_pct.columns:
    if coluna.startswith('QTD_'):  # Certifica-se de que estamos lidando apenas com as colunas relevantes
        coluna_percentual = coluna + '_%'
        proporcao_coluna = geral3_pct[coluna] / soma_total
        geral3_pct[coluna_percentual] = (proporcao_coluna * 100).round(2)

geral3_pct.head()

geral3 = geral3.merge(geral3_pct, on=['QTD_PUBLICA', 'QTD_PRIVADA','QTD_EXTERIOR'], how='outer')
geral3.head(13)

,NU_ANO,QTD_NAO_RESP,QTD_PUBLICA,QTD_PRIVADA,QTD_EXTERIOR,QTD_PUBLICA_%,QTD_PRIVADA_%,QTD_EXTERIOR_%
0,2011,752,188,60,0,75.81,24.19,0.00
1,2012,0,66,14,0,82.50,17.50,0.00
2,2013,0,155,24,0,86.59,13.41,0.00
3,2014,0,740,138,0,84.28,15.72,0.00
4,2015,884,52,20,0,72.22,27.78,0.00
5,2016,905,37,22,0,62.71,37.29,0.00
6,2017,881,78,19,4,77.23,18.81,3.96
7,2018,750,175,3,25,86.21,1.48,12.32
8,2019,778,199,1,0,99.50,0.50,0.00
9,2020,731,212,8,0,96.36,3.64,0.00


#### Media por tipo de escola.

In [10]:
geral3_media = df_enem.groupby('TP_ESCOLA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral3_media['TP_ESCOLA'] = geral3_media['TP_ESCOLA'].map({1: 'Não respondeu', 2: 'Escola Pública', 3: 'Escola Privada', 4: 'Exterior'})
geral3_media.head()

,TP_ESCOLA,MEDIA_OBJ,MEDIA_GLOBAL
0,Não respondeu,526.66,494.01
1,Escola Pública,510.65,444.94
2,Escola Privada,571.09,472.15
3,Exterior,598.47,613.81


#### Uni df ao geral

In [38]:
geral = geral.merge(geral3, on='NU_ANO', how='left')
geral.head()

,NU_ANO,QTD_INSCRITOS,ABS_ALL,ABS_ALL_%,QTD_INGLES,QTD_ESPANHOL,QTD_NAO_RESP,QTD_PUBLICA,QTD_PRIVADA,QTD_EXTERIOR
0,2018,953,209.0,21.93,426,527,750,175,3,25
1,2019,978,263.0,26.89,455,523,778,199,1,0


#### Quantidade por sexo
#### M - Masculino
#### F - Feminino

#### 3º Ajuste - base  ano de 2012
#### 0 = Masculino
#### 1 = Feminino

In [23]:
df_enem['TP_SEXO'] = df_enem['TP_SEXO'].replace({0: 'M', 1: 'F'})
df_enem[df_enem['NU_ANO'] == 2012]

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_CERTIFICADO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,TP_ANO_CONCLUIU,TP_ESCOLA,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050,Q051,Q052,Q053,Q054,Q055,Q056,Q057,Q058,Q059,Q060,Q061,Q062,Q063,Q064,Q065,Q066,Q067,Q068,Q069,Q070,Q071,Q072,Q073,Q074,Q075,TP_NACIONALIDADE,ANO_CONCLUIU,Q076,TP_REGIAO_PROVA,TP_REGIAO_ESC,ABS_DIA_1,ABS_DIA_2,ABS_ALL,FLAG_NULL_ESC,MEDIA_OBJ,MEDIA_GLOBAL
2000,400000000001,2012,13,M,1.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2933307,VITORIA DA CONQUISTA,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,NaN,NaN,NaN,NaN,I,B,D,3,A,B,A,A,D,A,D,D,A,D,D,B,A,A,A,D,A,B,4,5,0,5,5,5,5,E,D,A,D,C,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,1.0,1.0,1.0,1.0,NaN,NaN
2001,400000000002,2012,3,M,0.0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2611606,RECIFE,26,PE,1,1,1,1,141.0,137.0,146.0,150.0,537.7,552.2,532.6,468.3,1.0,P,80.0,60.0,60.0,80.0,0.0,280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,E,C,G,4,A,B,A,A,A,B,A,A,A,D,A,B,A,A,D,D,A,C,5,1,0,5,2,5,0,A,A,D,B,A,A,A,B,B,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,0.0,0.0,0.0,1.0,522.700,474.16
2002,400000000003,2012,5,F,0.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3301702,DUQUE DE CAXIAS,33,RJ,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,C,C,C,4,B,B,B,A,D,D,D,A,A,D,D,B,D,A,D,D,A,A,3,5,1,0,0,5,3,A,A,A,A,B,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sudeste,Sudeste,1.0,1.0,1.0,1.0,NaN,NaN
2003,400000000004,2012,11,M,0.0,3,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5300108,BRASILIA,53,DF,1,1,1,1,143.0,139.0,147.0,151.0,459.0,557.7,545.0,565.3,1.0,P,140.0,160.0,140.0,160.0,120.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,NaN,NaN,NaN,NaN,C,C,F,4,A,B,B,A,D,D,A,D,A,D,A,C,A,A,D,D,B,A,

#### 3º ajuste - Carregando agora a partir daqui pois o df foi ajustado. Notebook

In [35]:
df_enem.to_excel('C:\\Users\\Bruno\\PYTHON\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem4.xlsx', index=False)

#### Lendo de casa.

In [10]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem4.xlsx', sheet_name='Sheet1')

In [9]:
geral4 = df_enem.groupby(['NU_ANO', 'TP_SEXO'])['NU_INSCRICAO'].count().reset_index()

geral4 = geral4.pivot(index='NU_ANO', columns='TP_SEXO', values='NU_INSCRICAO').reset_index()
geral4.columns = ['NU_ANO', 'QTD_FEM','QTD_MASC']
geral4.head(13)

,NU_ANO,QTD_FEM,QTD_MASC
0,2010,571,429
1,2011,556,444
2,2012,545,455
3,2013,574,426
4,2014,575,425
5,2015,534,422
6,2016,538,426
7,2017,559,423
8,2018,613,340
9,2019,634,344


#### Proporção Sexo

In [18]:
geral4_pct = geral4[['QTD_FEM','QTD_MASC']]
geral4_pct.head(13)

,QTD_FEM,QTD_MASC
0,571,429
1,556,444
2,545,455
3,574,426
4,575,425
5,534,422
6,538,426
7,559,423
8,613,340
9,634,344


In [19]:
soma_total = geral4_pct.sum(axis=1)

# Iterar sobre as colunas e criar as colunas percentuais automaticamente
for coluna in geral4_pct.columns:
    if coluna.startswith('QTD_'):  # Certifica-se de que estamos lidando apenas com as colunas relevantes
        coluna_percentual = coluna + '_%'
        proporcao_coluna = geral4_pct[coluna] / soma_total
        geral4_pct[coluna_percentual] = (proporcao_coluna * 100).round(2)

geral4_pct.head(13)

,QTD_FEM,QTD_MASC,QTD_FEM_%,QTD_MASC_%
0,571,429,57.10,42.90
1,556,444,55.60,44.40
2,545,455,54.50,45.50
3,574,426,57.40,42.60
4,575,425,57.50,42.50
5,534,422,55.86,44.14
6,538,426,55.81,44.19
7,559,423,56.92,43.08
8,613,340,64.32,35.68
9,634,344,64.83,35.17


In [20]:
geral4 = geral4.merge(geral4_pct, on=['QTD_MASC', 'QTD_FEM'], how='outer')
geral4.head(13)

,NU_ANO,QTD_FEM,QTD_MASC,QTD_FEM_%,QTD_MASC_%
0,2010,571,429,57.10,42.90
1,2011,556,444,55.60,44.40
2,2012,545,455,54.50,45.50
3,2013,574,426,57.40,42.60
4,2014,575,425,57.50,42.50
5,2015,534,422,55.86,44.14
6,2016,538,426,55.81,44.19
7,2017,559,423,56.92,43.08
8,2018,613,340,64.32,35.68
9,2019,634,344,64.83,35.17


#### Média por genêro.

In [12]:
geral4_media = df_enem.groupby('TP_SEXO')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral4_media['TP_SEXO'] = geral4_media['TP_SEXO'].map({'F': 'Feminino', 'M': 'Masculino'})
geral4_media.head()

,TP_SEXO,MEDIA_OBJ,MEDIA_GLOBAL
0,Feminino,514.12,455.22
1,Masculino,536.73,467.84


#### Mesclar caso queira unir o df geral

In [ ]:
geral = geral.merge(geral4, on='NU_ANO', how='left')
geral.head()

### Tipo de adminstração
#### 1 - FEDERAL
#### 2 - ESTADUAL
#### 3 - MUNICIPAL
#### 4 - PRIVADA

In [31]:
geral5 = df_enem.groupby(['NU_ANO', 'TP_DEPENDENCIA_ADM_ESC'])['NU_INSCRICAO'].count().reset_index()
geral5 = geral5.pivot(index='NU_ANO', columns='TP_DEPENDENCIA_ADM_ESC', values='NU_INSCRICAO').reset_index()
geral5.columns = ['NU_ANO', 'QTD_FEDERAL', 'QTD_ESTADUAL', 'QTD_MUNICIPAL', 'QTD_PRIVADA']
geral5 = geral5.fillna(0).astype(int) #Possui dados faltantes
geral5.head(13)

,NU_ANO,QTD_FEDERAL,QTD_ESTADUAL,QTD_MUNICIPAL,QTD_PRIVADA
0,2010,3,94,1,22
1,2011,6,35,2,17
2,2012,3,63,1,14
3,2013,1,150,6,24
4,2014,3,72,5,12
5,2015,2,49,1,20
6,2016,0,36,1,22
7,2017,9,69,0,23
8,2018,15,175,3,25
9,2019,20,153,0,3


#### Irei fazer a proporção com os dados disponiveis e não com qtd de inscritos totais, pois tem poucos dados disponiveis

In [32]:
geral5['TOTAL'] = geral5['QTD_FEDERAL'] + geral5['QTD_ESTADUAL'] + geral5['QTD_MUNICIPAL'] + geral5['QTD_PRIVADA']

for tipo in ['FEDERAL', 'ESTADUAL', 'MUNICIPAL', 'PRIVADA']:
    coluna_qtd = f'QTD_{tipo}'
    coluna_percentual = f'QTD_{tipo}_%'

    geral5[coluna_percentual] = ((geral5[coluna_qtd] / geral5['TOTAL']) * 100).round(2)

geral5.head(13)

,NU_ANO,QTD_FEDERAL,QTD_ESTADUAL,QTD_MUNICIPAL,QTD_PRIVADA,TOTAL,QTD_FEDERAL_%,QTD_ESTADUAL_%,QTD_MUNICIPAL_%,QTD_PRIVADA_%
0,2010,3,94,1,22,120,2.50,78.33,0.83,18.33
1,2011,6,35,2,17,60,10.00,58.33,3.33,28.33
2,2012,3,63,1,14,81,3.70,77.78,1.23,17.28
3,2013,1,150,6,24,181,0.55,82.87,3.31,13.26
4,2014,3,72,5,12,92,3.26,78.26,5.43,13.04
5,2015,2,49,1,20,72,2.78,68.06,1.39,27.78
6,2016,0,36,1,22,59,0.00,61.02,1.69,37.29
7,2017,9,69,0,23,101,8.91,68.32,0.00,22.77
8,2018,15,175,3,25,218,6.88,80.28,1.38,11.47
9,2019,20,153,0,3,176,11.36,86.93,0.00,1.70


#### Média por tipo de dependência administrativa.

In [13]:
geral5_media = df_enem.groupby('TP_DEPENDENCIA_ADM_ESC')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral5_media['TP_DEPENDENCIA_ADM_ESC'] = geral5_media['TP_DEPENDENCIA_ADM_ESC'].map({1: 'Federal', 2: 'Estadual', 3: 'Municipal', 4: 'Privada'})
geral5_media.head()

,TP_DEPENDENCIA_ADM_ESC,MEDIA_OBJ,MEDIA_GLOBAL
0,Federal,589.22,614.57
1,Estadual,507.54,488.51
2,Municipal,483.97,438.30
3,Privada,591.63,593.59


#### Caso queira unir os df em um só.

In [ ]:
geral = geral.merge(geral5, on='NU_ANO', how='left')
geral.head()

#### Cor e Raça
#### 0 - Não declarado
#### 1 - Branca
#### 2 - Preta 
#### 3 - Parda
#### 4 - Amarela
#### 5 - Indígena
#### 6 - Não dispõe da informação
#### Bases de 2015, 2016, 2021 e 2022 tem os itens "6 - Não dispõe da informação".

In [19]:
geral6 = df_enem.groupby(['NU_ANO', 'TP_COR_RACA'])['NU_INSCRICAO'].count().reset_index()


geral6 = geral6.pivot(index='NU_ANO', columns='TP_COR_RACA', values='NU_INSCRICAO').reset_index()

# Renomear as colunas para facilitar a compreensão
geral6.columns = ['NU_ANO', 'QTD_NAO_DECLARADO','QTD_BRANCA','QTD_PRETA','QTD_PARDA','QTD_AMARELA','QTD_INDIGENA','QTD_NAO_DISPOE']
geral6 = geral6.fillna(0).astype(int)
geral6.head(13)

,NU_ANO,QTD_NAO_DECLARADO,QTD_BRANCA,QTD_PRETA,QTD_PARDA,QTD_AMARELA,QTD_INDIGENA,QTD_NAO_DISPOE
0,2010,29,418,122,404,23,4,0
1,2011,30,423,135,376,31,5,0
2,2012,21,417,126,397,32,7,0
3,2013,15,411,105,443,20,6,0
4,2014,16,390,132,428,29,5,0
5,2015,18,429,128,360,17,3,1
6,2016,12,447,107,367,25,2,4
7,2017,21,375,155,411,17,3,0
8,2018,21,300,131,477,21,3,0
9,2019,13,309,152,470,26,8,0


#### Proporções

In [20]:
geral6_pct = geral6[['QTD_NAO_DECLARADO','QTD_BRANCA','QTD_PRETA','QTD_PARDA','QTD_AMARELA','QTD_INDIGENA','QTD_NAO_DISPOE']]
geral6_pct.head(13)

,QTD_NAO_DECLARADO,QTD_BRANCA,QTD_PRETA,QTD_PARDA,QTD_AMARELA,QTD_INDIGENA,QTD_NAO_DISPOE
0,29,418,122,404,23,4,0
1,30,423,135,376,31,5,0
2,21,417,126,397,32,7,0
3,15,411,105,443,20,6,0
4,16,390,132,428,29,5,0
5,18,429,128,360,17,3,1
6,12,447,107,367,25,2,4
7,21,375,155,411,17,3,0
8,21,300,131,477,21,3,0
9,13,309,152,470,26,8,0


In [21]:
soma_total = geral6_pct.sum(axis=1)

# Iterar sobre as colunas e criar as colunas percentuais automaticamente
for coluna in geral6_pct.columns:
    if coluna.startswith('QTD_'):  # Certifica-se de que estamos lidando apenas com as colunas relevantes
        coluna_percentual = coluna + '_%'
        proporcao_coluna = geral6_pct[coluna] / soma_total
        geral6_pct[coluna_percentual] = (proporcao_coluna * 100).round(2)

geral6_pct.head()

geral6 = geral6.merge(geral6_pct, on=['QTD_NAO_DECLARADO','QTD_BRANCA','QTD_PRETA','QTD_PARDA','QTD_AMARELA','QTD_INDIGENA','QTD_NAO_DISPOE'], how='outer')
geral6.head(13)

,NU_ANO,QTD_NAO_DECLARADO,QTD_BRANCA,QTD_PRETA,QTD_PARDA,QTD_AMARELA,QTD_INDIGENA,QTD_NAO_DISPOE,QTD_NAO_DECLARADO_%,QTD_BRANCA_%,QTD_PRETA_%,QTD_PARDA_%,QTD_AMARELA_%,QTD_INDIGENA_%,QTD_NAO_DISPOE_%
0,2010,29,418,122,404,23,4,0,2.90,41.80,12.20,40.40,2.30,0.40,0.00
1,2011,30,423,135,376,31,5,0,3.00,42.30,13.50,37.60,3.10,0.50,0.00
2,2012,21,417,126,397,32,7,0,2.10,41.70,12.60,39.70,3.20,0.70,0.00
3,2013,15,411,105,443,20,6,0,1.50,41.10,10.50,44.30,2.00,0.60,0.00
4,2014,16,390,132,428,29,5,0,1.60,39.00,13.20,42.80,2.90,0.50,0.00
5,2015,18,429,128,360,17,3,1,1.88,44.87,13.39,37.66,1.78,0.31,0.10
6,2016,12,447,107,367,25,2,4,1.24,46.37,11.10,38.07,2.59,0.21,0.41
7,2017,21,375,155,411,17,3,0,2.14,38.19,15.78,41.85,1.73,0.31,0.00
8,2018,21,300,131,477,21,3,0,2.20,31.48,13.75,50.05,2.20,0.31,0.00
9,2019,13,309,152,470,26,8,0,1.33,31.60,15.54,48.06,2.66,0.82,0.00


#### Média por Cor e Raça

In [18]:
geral6_media = df_enem.groupby('TP_COR_RACA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral6_media['TP_COR_RACA'] = geral6_media['TP_COR_RACA'].map({0: 'Não declarado', 1: 'Branca', 2: 'Preta', 3: 'Parda', 4: 'Amarela', 5: 'Indígena', 6: 'Não dispões da informação'})
geral6_media.head(7)

,TP_COR_RACA,MEDIA_OBJ,MEDIA_GLOBAL
0,Não declarado,541.67,468.14
1,Branca,540.22,469.18
2,Preta,509.91,445.87
3,Parda,512.74,457.35
4,Amarela,516.50,452.70
5,Indígena,483.90,437.76
6,Não dispões da informação,546.98,415.19


#### Estado Civil - 2011, 2017, 2019, 2020, 2021 e 2022.
#### 0 - Não informado
#### 1 - Solteiro
#### 2 - Casado
#### 3 - Divorciado
#### 4 - Viúvo

#### 2010, 2012 a 2016, 2018
#### 0 - Solteiro
#### 1 - Casado
#### 2 - Divorciado
#### 3 - Viúvo

#### 5º Ajuste - Bases diferentes.


In [33]:
df_enem.loc[df_enem['NU_ANO'].between(2012, 2016), 'TP_ESTADO_CIVIL'] = df_enem['TP_ESTADO_CIVIL'] + 1
df_enem.loc[df_enem['NU_ANO'].isin([2010, 2018]), 'TP_ESTADO_CIVIL'] = df_enem['TP_ESTADO_CIVIL'] + 1
df_enem[df_enem['NU_ANO'] == 2012]

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_CERTIFICADO,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q01,Q02,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35,Q36,Q37,Q38,Q39,Q40,Q41,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57,TP_ANO_CONCLUIU,TP_ESCOLA,NO_ENTIDADE_CERTIFICACAO,CO_UF_ENTIDADE_CERTIFICACAO,SG_UF_ENTIDADE_CERTIFICACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,Q028,Q029,Q030,Q031,Q032,Q033,Q034,Q035,Q036,Q037,Q038,Q039,Q040,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050,Q051,Q052,Q053,Q054,Q055,Q056,Q057,Q058,Q059,Q060,Q061,Q062,Q063,Q064,Q065,Q066,Q067,Q068,Q069,Q070,Q071,Q072,Q073,Q074,Q075,TP_NACIONALIDADE,ANO_CONCLUIU,Q076,TP_REGIAO_PROVA,TP_REGIAO_ESC,ABS_DIA_1,ABS_DIA_2,ABS_ALL,FLAG_NULL_ESC,MEDIA_OBJ,MEDIA_GLOBAL
2000,400000000001,2012,13,M,2.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2933307,VITORIA DA CONQUISTA,29,BA,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,NaN,NaN,NaN,NaN,I,B,D,3,A,B,A,A,D,A,D,D,A,D,D,B,A,A,A,D,A,B,4,5,0,5,5,5,5,E,D,A,D,C,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,1.0,1.0,1,1,NaN,NaN
2001,400000000002,2012,3,M,1.0,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2611606,RECIFE,26,PE,1,1,1,1,141.0,137.0,146.0,150.0,537.7,552.2,532.6,468.3,1.0,P,80.0,60.0,60.0,80.0,0.0,280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,E,C,G,4,A,B,A,A,A,B,A,A,A,D,A,B,A,A,D,D,A,C,5,1,0,5,2,5,0,A,A,D,B,A,A,A,B,B,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nordeste,Nordeste,0.0,0.0,0,1,522.700,474.16
2002,400000000003,2012,5,F,1.0,3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3301702,DUQUE DE CAXIAS,33,RJ,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,NaN,NaN,NaN,NaN,C,C,C,4,B,B,B,A,D,D,D,A,A,D,D,B,D,A,D,D,A,A,3,5,1,0,0,5,3,A,A,A,A,B,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sudeste,Sudeste,1.0,1.0,1,1,NaN,NaN
2003,400000000004,2012,11,M,1.0,3,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5300108,BRASILIA,53,DF,1,1,1,1,143.0,139.0,147.0,151.0,459.0,557.7,545.0,565.3,1.0,P,140.0,160.0,140.0,160.0,120.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,NaN,NaN,NaN,NaN,C,C,F,4,A,B,B,A,D,D,A,D,A,D,A,C,A,A,D,D,B,A,5,5,0,5,5,5,

#### 5º Ajuste - Salvando nova alteração de dataset, ajustando os anos diferentes. Mais recente. Notebook.

In [34]:
df_enem.to_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem6.xlsx', index=False)

#### 5º Ajuste - Carregando a partir daqui. Mais recente. Notebook.

In [ ]:
df_enem = pd.read_excel('C:\\Users\\Bruno\\PYTHON\\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem6.xlsx')

In [4]:
df_enem = pd.read_excel('D:\Repositório GIT\Projetos\Analise-de-Dados-ENEM\Datasets\Teste\enem6.xlsx')

#### Trabalhando...

In [36]:
geral7 = df_enem.groupby(['NU_ANO', 'TP_ESTADO_CIVIL'])['NU_INSCRICAO'].count().reset_index()


geral7 = geral7.pivot(index='NU_ANO', columns='TP_ESTADO_CIVIL', values='NU_INSCRICAO').reset_index()

# Renomear as colunas para facilitar a compreensão
geral7.columns = ['NU_ANO', 'QTD_NAO_INFORMADO','QTD_SOLTEIRO','QTD_CASADO','QTD_DIVORCIADO','QTD_VIUVO']
geral7 = geral7.fillna(0).astype(int) #Possui NAn
geral7.head(13)

,NU_ANO,QTD_NAO_INFORMADO,QTD_SOLTEIRO,QTD_CASADO,QTD_DIVORCIADO,QTD_VIUVO
0,2010,0,765,214,18,3
1,2011,0,788,182,29,1
2,2012,0,769,206,22,3
3,2013,0,797,181,21,1
4,2014,0,758,212,28,2
5,2015,0,765,123,21,3
6,2016,0,787,104,23,3
7,2017,36,781,139,25,1
8,2018,0,825,76,14,2
9,2019,36,781,130,30,1


#### Proporção.

In [37]:
geral7_pct = geral7[['QTD_NAO_INFORMADO','QTD_SOLTEIRO','QTD_CASADO','QTD_DIVORCIADO','QTD_VIUVO']]
geral7_pct.head(13)

,QTD_NAO_INFORMADO,QTD_SOLTEIRO,QTD_CASADO,QTD_DIVORCIADO,QTD_VIUVO
0,0,765,214,18,3
1,0,788,182,29,1
2,0,769,206,22,3
3,0,797,181,21,1
4,0,758,212,28,2
5,0,765,123,21,3
6,0,787,104,23,3
7,36,781,139,25,1
8,0,825,76,14,2
9,36,781,130,30,1


In [38]:
soma_total = geral7_pct.sum(axis=1)

# Iterar sobre as colunas e criar as colunas percentuais automaticamente
for coluna in geral7_pct.columns:
    if coluna.startswith('QTD_'):  # Certifica-se de que estamos lidando apenas com as colunas relevantes
        coluna_percentual = coluna + '_%'
        proporcao_coluna = geral7_pct[coluna] / soma_total
        geral7_pct[coluna_percentual] = (proporcao_coluna * 100).round(2)

geral7_pct.head()

geral7 = geral7.merge(geral7_pct, on=['QTD_NAO_INFORMADO','QTD_SOLTEIRO','QTD_CASADO','QTD_DIVORCIADO','QTD_VIUVO'], how='outer')
geral7.head(13)

,NU_ANO,QTD_NAO_INFORMADO,QTD_SOLTEIRO,QTD_CASADO,QTD_DIVORCIADO,QTD_VIUVO,QTD_NAO_INFORMADO_%,QTD_SOLTEIRO_%,QTD_CASADO_%,QTD_DIVORCIADO_%,QTD_VIUVO_%
0,2010,0,765,214,18,3,0.00,76.50,21.40,1.80,0.30
1,2011,0,788,182,29,1,0.00,78.80,18.20,2.90,0.10
2,2012,0,769,206,22,3,0.00,76.90,20.60,2.20,0.30
3,2013,0,797,181,21,1,0.00,79.70,18.10,2.10,0.10
4,2014,0,758,212,28,2,0.00,75.80,21.20,2.80,0.20
5,2015,0,765,123,21,3,0.00,83.88,13.49,2.30,0.33
6,2016,0,787,104,23,3,0.00,85.82,11.34,2.51,0.33
7,2017,36,781,139,25,1,3.67,79.53,14.15,2.55,0.10
8,2018,0,825,76,14,2,0.00,89.97,8.29,1.53,0.22
9,2019,36,781,130,30,1,3.68,79.86,13.29,3.07,0.10


#### Média por Estado Civil.

In [19]:
geral7_media = df_enem.groupby('TP_ESTADO_CIVIL')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral7_media['TP_ESTADO_CIVIL'] = geral7_media['TP_ESTADO_CIVIL'].map({0: 'Não informado', 1: 'Solteiro(a)', 2: 'Casado(a)', 3: 'Divorciado(a)', 4: 'Viúvo(a)'})
geral7_media.head(6)

,TP_ESTADO_CIVIL,MEDIA_OBJ,MEDIA_GLOBAL
0,Não informado,508.12,511.02
1,Solteiro(a),524.51,473.82
2,Casado(a),516.74,382.87
3,Divorciado(a),514.38,420.30
4,Viúvo(a),475.41,389.42


### Região geográfica

In [16]:
geral8_media = df_enem.groupby('TP_REGIAO_PROVA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral8_media.head(6)

,TP_REGIAO_PROVA,MEDIA_OBJ,MEDIA_GLOBAL
0,Centro-Oeste,510.03,457.97
1,DF,515.36,403.98
2,Nordeste,515.29,469.58
3,Norte,500.58,448.62
4,Sudeste,536.95,458.22
5,Sul,532.23,459.72


#### Códigos adaptados para municipios e uf

In [13]:
municipality_view = ['TP_REGIAO_PROVA','SG_UF_PROVA', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA']
uf_view           = ['TP_REGIAO_PROVA', 'SG_UF_PROVA']
list_metrics      = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}

# TP_ANO_CONCLUIU, TP_ESTADO_CIVIL

cat_columns       = [
    'TP_FAIXA_ETARIA', 'TP_SEXO',
    'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
    'TP_ESCOLA', 'TP_ENSINO',  'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_PRESENCA_CN',
    'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 
]

In [14]:
# DATA PREP
notas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()
lista_provas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()

In [10]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    
def get_count(df_enem, cols_to_group, column, column_name = None):
    agg_result = (df_enem
         .groupby(cols_to_group)
         .agg({column:'count'})
         .rename(columns = {column : column_name} )
        )
    return agg_result

def build_statistics_metrics(ano_enem : int, 
                             lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(ano_enem, lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(ano_enem, lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df

def build_statistics_metrics(lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df


In [8]:
geral8 = get_count(df_enem, municipality_view, 'NU_INSCRICAO', 'QUANTIDADE_INSCRITOS')

In [9]:
geral8.to_excel('C:\\Users\\Bruno\\PYTHON\PROJETOS\\Analise-de-Dados-ENEM\\Datasets\\Teste\\count.xlsx', index=False)

In [ ]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    
    
    
def build_statistics_metrics(ano_enem : int, 
                             lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(ano_enem, lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(ano_enem, lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df


def build_distribution_freq(df_enem, cols_to_group, cat_columns):
    
    def get_distribution_freq(df_enem, cols_to_group, cat_column):
        agg_result = (pd.get_dummies(df_enem[cols_to_group+[cat_column]],
               columns = [cat_column])
                 .groupby(cols_to_group).sum()
                )
        return agg_result
    
    result    = [get_distribution_freq(df_enem, cols_to_group, cat_column) for cat_column in cat_columns]
    result_df = (pd.concat(result, axis = 1))
    return result_df

def get_count(ano_enem, df_enem, cols_to_group, column, column_name = None):
    agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group)
         .agg({column:'count'})
         .rename(columns = {column : column_name} )
        )
    return agg_result

### Unidade da Federação - UF

In [20]:
geral9_media = df_enem.groupby('SG_UF_PROVA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral9_media.head(27)

,SG_UF_PROVA,MEDIA_OBJ,MEDIA_GLOBAL
0,AC,487.42,465.39
1,AL,494.99,450.58
2,AM,492.36,427.85
3,AP,486.18,420.06
4,BA,508.58,453.21
5,CE,533.46,486.84
6,DF,515.36,403.98
7,ES,535.21,479.81
8,GO,515.07,475.74
9,MA,499.60,470.71


In [21]:
get_count(df_enem, uf_view, 'NU_INSCRICAO', 'QUANTIDADE_INSCRITOS')

QUANTIDADE_INSCRITOS
TP_REGIAO_PROVA SG_UF_PROVA                      
Centro-Oeste    GO                            430
                MS                            187
                MT                            224
DF              DF                            290
Nordeste        AL                            185
                BA                           1143
                CE                            855
                MA                            400
                PB                            335
                PE                            685
                PI                            240
                RN                            342
                SE                            197
Norte           AC                             53
                AM                            214
                AP                             45
                PA                            554
                RO                             92
                RR                             31
                TO                            102
Sudeste         ES                            284
                MG                           1312
                RJ                            898
                SP                           2248
Sul             PR                            521
                RS                            597
                SC                            207

In [22]:
# Não possui as MEDIA_OBJ E MEDIA_GLOBAL
build_statistics_metrics(lista_provas, df_enem, uf_view, list_metrics)

,TP_REGIAO_PROVA,SG_UF_PROVA,MIN_NOTA_CN,MIN_NOTA_CH,MIN_NOTA_LC,MIN_NOTA_MT,MIN_NOTA_REDACAO,AVG_NOTA_CN,AVG_NOTA_CH,AVG_NOTA_LC,AVG_NOTA_MT,AVG_NOTA_REDACAO,MED_NOTA_CN,MED_NOTA_CH,MED_NOTA_LC,MED_NOTA_MT,MED_NOTA_REDACAO,MAX_NOTA_CN,MAX_NOTA_CH,MAX_NOTA_LC,MAX_NOTA_MT,MAX_NOTA_REDACAO,STD_NOTA_CN,STD_NOTA_CH,STD_NOTA_LC,STD_NOTA_MT,STD_NOTA_REDACAO
0,Centro-Oeste,GO,303.6,334.5,323.7,317.3,0.0,491.408442,538.273333,515.016340,516.913378,521.811594,487.30,539.10,517.65,501.00,560.0,871.3,859.1,686.2,923.2,980.0,81.126703,82.961026,73.984821,108.224700,246.462429
1,Centro-Oeste,MS,320.6,318.9,335.2,318.8,0.0,474.021324,531.380714,509.386567,502.997692,437.423313,471.80,542.45,513.95,488.85,480.0,770.2,749.5,657.6,873.2,880.0,71.578764,81.307889,70.681906,100.407735,247.092573
2,Centro-Oeste,MT,329.9,365.5,333.4,325.9,0.0,482.023899,534.402469,512.334810,505.041290,465.594595,475.10,539.30,522.20,482.80,520.0,755.1,764.6,706.2,858.5,920.0,78.416465,87.609034,72.052739,104.471753,251.293767
3,DF,DF,357.9,350.0,357.7,316.7,0.0,489.562722,548.976136,529.440116,496.203030,411.938326,478.10,554.05,532.60,471.50,480.0,750.8,724.2,761.3,870.5,940.0,73.122066,72.111624,64.008513,105.193227,270.851632
4,Nordeste,AL,339.0,326.3,326.6,332.9,0.0,473.018797,513.999270,498.704511,495.795349,489.836601,461.80,512.80,499.90,471.70,540.0,673.7,771.1,688.3,827.2,980.0,75.694486,84.559604,75.475228,108.332237,250.532603
5,Nordeste,BA,333.6,314.3,295.3,0.0,0.0,485.645443,533.570885,514.097875,502.320000,482.056277,477.80,537.00,520.25,481.80,540.0,716.4,774.8,685.4,869.0,960.0,72.990632,83.203839,68.188947,106.521012,253.924117
6,Nordeste,CE,335.5,329.7,310.9,298.1,0.0,504.680395,555.990806,533.775618,541.895118,530.883212,501.70,564.45,544.80,529.20,580.0,753.7,798.9,723.4,903.9,1000.0,80.658374,86.183012,72.805593,119.562690,255.999334
7,Nordeste,MA,283.7,0.0,0.0,292.0,0.0,479.453356,523.128431,504.392208,498.782667,521.428571,478.50,531.05,514.30,486.40,560.0,661.6,739.9,724.9,825.2,1000.0,69.954846,94.703680,86.421683,102.290395,228.373315
8,Nordeste,PB,313.3,328.9,298.0,291.9,0.0,491.521401,539.627376,521.801163,517.417063,518.357639,486.20,548.40,524.30,502.55,560.0,727.2,732.4,721.7,850.0,980.0,74.362441,83.658232,72.462547,115.264205,244.752112
9,Nordeste,PE,286.4,297.2,320.7,294.4,0.0,493.038821,537.913121,526.362298,524.487423,510.702128,487.95,539.60,528.00,508.10,560.0,743.9,772.2,706.6,882.0,980.0,82.227390,85.416729,68.462919,115.194230,252.653442


### Municípios - PRECISA AJUSTAR A BASE

In [ ]:
geral10_media = df_enem.groupby('SG_UF_PROVA')[['MEDIA_OBJ', 'MEDIA_GLOBAL']].mean().reset_index().round(2)
geral10_media.head(27)

In [17]:
get_count(df_enem, municipality_view, 'NU_INSCRICAO', 'QUANTIDADE_INSCRITOS')

QUANTIDADE_INSCRITOS
TP_REGIAO_PROVA SG_UF_PROVA CO_MUNICIPIO_PROVA NO_MUNICIPIO_PROVA                         
Centro-Oeste    GO          5200258            AGUAS LINDAS DE GOIAS                     4
                                               Águas Lindas de Goiás                     5
                            5200308            Alexânia                                  1
                            5201108            ANAPOLIS                                 12
                                               Anápolis                                 16
...                                                                                    ...
Sul             SC          4219309            Videira                                   2
                            4219507            XANXERE                                   2
                                               Xanxerê                                   1
                            4219705            XAXIM                                     1
                                               Xaxim                                     1

[2150 rows x 1 columns]

In [18]:
# Não possui as MEDIA_OBJ E MEDIA_GLOBAL
build_statistics_metrics(lista_provas, df_enem, municipality_view, list_metrics)

,TP_REGIAO_PROVA,SG_UF_PROVA,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,MIN_NOTA_CN,MIN_NOTA_CH,MIN_NOTA_LC,MIN_NOTA_MT,MIN_NOTA_REDACAO,AVG_NOTA_CN,AVG_NOTA_CH,AVG_NOTA_LC,AVG_NOTA_MT,AVG_NOTA_REDACAO,MED_NOTA_CN,MED_NOTA_CH,MED_NOTA_LC,MED_NOTA_MT,MED_NOTA_REDACAO,MAX_NOTA_CN,MAX_NOTA_CH,MAX_NOTA_LC,MAX_NOTA_MT,MAX_NOTA_REDACAO,STD_NOTA_CN,STD_NOTA_CH,STD_NOTA_LC,STD_NOTA_MT,STD_NOTA_REDACAO
0,Centro-Oeste,GO,5200258,AGUAS LINDAS DE GOIAS,394.3,378.7,364.2,342.2,0.0,418.750000,455.750000,418.000000,383.800000,55.000000,418.75,455.75,418.0,383.80,0.0,443.2,532.8,471.8,425.4,220.0,34.577522,108.965155,76.084690,58.831284,110.000000
1,Centro-Oeste,GO,5200258,Águas Lindas de Goiás,426.9,407.2,329.0,392.4,0.0,448.366667,487.025000,460.800000,426.200000,415.000000,450.30,496.10,467.5,410.80,490.0,467.9,548.7,579.2,475.4,680.0,20.568260,67.143050,116.749732,43.590366,304.357246
2,Centro-Oeste,GO,5200308,Alexânia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Centro-Oeste,GO,5201108,ANAPOLIS,352.7,430.8,426.8,317.3,0.0,483.288889,538.211111,522.488889,508.800000,426.000000,459.80,523.60,498.2,524.50,420.0,608.1,665.4,608.9,660.0,775.0,81.181701,87.027071,61.272311,114.105171,206.529524
4,Centro-Oeste,GO,5201108,Anápolis,474.2,432.0,333.9,439.6,0.0,546.758333,555.733333,540.941667,579.316667,636.666667,551.75,559.95,576.2,559.95,670.0,610.3,709.0,665.8,782.2,940.0,46.167156,90.423442,96.839253,108.012498,261.996993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,Sul,SC,4219309,Videira,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2146,Sul,SC,4219507,XANXERE,423.4,561.0,480.3,567.0,720.0,423.400000,561.000000,480.300000,567.000000,720.000000,423.40,561.00,480.3,567.00,720.0,423.4,561.0,480.3,567.0,720.0,NaN,NaN,NaN,NaN,NaN
2147,Sul,SC,4219507,Xanxerê,413.4,431.2,500.7,493.4,680.0,413.400000,431.200000,500.700000,493.400000,680.000000,413.40,431.20,500.7,493.40,680.0,413.4,431.2,500.7,493.4,680.0,NaN,NaN,NaN,NaN,NaN
2148,Sul,SC,4219705,XAXIM,529.2,480.7,450.8,427.2,500.0,529.200000,480.700000,450.800000,427.200000,500.000000,529.20,480.70,450.8,427.20,500.0,529.2,480.7,450.8,427.2,500.0,NaN,NaN,NaN,NaN,NaN
